In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from classifierModel import EncoderModel, ClassifierModel
import torch
import numpy as np
import os
import pandas as pd
import spacy
import sys
import torch.nn as nn
import torch.nn.functional as F
import pickle

from data_utils import (IndexVectorizer,
                        SpacyTokenizer,
                        TextDataset,
                        LMDataLoader,
                        CLFDataLoader)

In [3]:
BATCH_SIZE = 300

# GPU setup
use_gpu = torch.cuda.is_available()
device_num = 0
device = torch.device(f"cuda:{device_num}" if use_gpu else "cpu")
print(device)

cuda:0


In [4]:
DATA_DIR = '../data/imdb/'
TRAIN_PATH = os.path.join(DATA_DIR, 'train.csv')
VALID_PATH = os.path.join(DATA_DIR, 'valid.csv')

train = pd.read_csv(TRAIN_PATH)
valid = pd.read_csv(VALID_PATH)

vectorizer = pickle.load(open("lm_vectorizer.pkl", "rb"))
train_ds = TextDataset(data=train, vectorizer=vectorizer, 
                       text_col='text', label_col='label')
valid_ds = TextDataset(data=valid, vectorizer=vectorizer, 
                       text_col='text', label_col='label')

In [5]:
train_dl = CLFDataLoader(dataset=train_ds, batch_size=BATCH_SIZE)
valid_dl = CLFDataLoader(dataset=valid_ds, batch_size=BATCH_SIZE)
# Test the iterators
train_it = iter(train_dl)
valid_it = iter(valid_dl)
x, y = next(train_it)

In [6]:
#print(x.shape)
#print(x[0].shape)
#print(y.shape)
#print(y[0])

d = torch.load(DATA_DIR+"models/LM__2019-01-31.json")
for i in d:
    print(i)
del d["decoder.weight"]
del d["decoder.bias"]

encoder.weight
lstm1.weight_ih_l0
lstm1.weight_hh_l0
lstm1.bias_ih_l0
lstm1.bias_hh_l0
lstm2.weight_ih_l0
lstm2.weight_hh_l0
lstm2.bias_ih_l0
lstm2.bias_hh_l0
lstm3.weight_ih_l0
lstm3.weight_hh_l0
lstm3.bias_ih_l0
lstm3.bias_hh_l0
decoder.weight
decoder.bias


In [7]:
m = EncoderModel(vectorizer)
for i in m.state_dict():
    print(i)
m.load_state_dict(d)
m.requires_grad = False

encoder.weight
lstm1.weight_ih_l0
lstm1.weight_hh_l0
lstm1.bias_ih_l0
lstm1.bias_hh_l0
lstm2.weight_ih_l0
lstm2.weight_hh_l0
lstm2.bias_ih_l0
lstm2.bias_hh_l0
lstm3.weight_ih_l0
lstm3.weight_hh_l0
lstm3.bias_ih_l0
lstm3.bias_hh_l0


In [8]:
c = ClassifierModel()

final = nn.Sequential(m,c)
if use_gpu:
    final = final.to(device)

#finalOutput = final.forward(x)
#print("Shape of final outputs: ",finalOutput.shape)
#print("Final outputs: ",finalOutput)

In [ ]:
optimizer = torch.optim.SGD(final.parameters(), lr = 0.01)
criterion = nn.CrossEntropyLoss()

n_epochs = 100

for epoch in range(n_epochs):
    i = 0
    accuracies = []
    for x, y in train_dl:
        x = x.to(device)
        y = y.to(device)
        i += 1
        final.zero_grad()
        res = final(x)
        error = criterion(res, y)
        error.backward()
        optimizer.step()
        if i % 50 == 0:
            pred_class = torch.argmax(res, dim=1)
            errors = pred_class == y
            accuracy = torch.mean(errors.type(torch.float))
            accuracies.append(accuracy.item())
            mean_acc = np.mean(accuracies)
            print(f'Batch: {i} Mean Accuracy: {mean_acc}')
    

Batch: 50 Mean Accuracy: 0.5766666531562805
